In [1]:
import pandas as pd
import os
import glob
from tqdm import tqdm
import warnings

# Configuração para ignorar avisos irrelevantes
warnings.filterwarnings("ignore")

# =============================================================================
# 1. CONFIGURAÇÕES
# =============================================================================
BASE_DIR = r"E:\Estudos_Cybersecurity\Datasets_Cybersecurity"
OUTPUT_FILE = os.path.join(BASE_DIR, "dataset_treino_final.parquet")

# Fração de amostragem (0.1 = 10% dos dados)
# Isso é suficiente para criar um modelo robusto sem travar a memória RAM
SAMPLE_RATE = 0.10 

# Mapeamento para corrigir nomes de colunas diferentes entre 2017 e 2019
column_mapping = {
    "Flow ID": "Flow_ID",
    "Source IP": "Source_IP",
    "Source Port": "Source_Port",
    "Destination IP": "Destination_IP",
    "Destination Port": "Destination_Port",
    "Protocol": "Protocol",
    "Timestamp": "Timestamp",
    # Adicione outros se necessário, mas esses são os principais identificadores
}

# =============================================================================
# 2. FUNÇÃO DE LEITURA E AMOSTRAGEM
# =============================================================================
def carregar_e_amostrar(diretorio_raiz):
    dfs = []
    
    # Busca recursiva por todos os arquivos .parquet
    arquivos = glob.glob(os.path.join(diretorio_raiz, "**", "*.parquet"), recursive=True)
    
    # Remove o arquivo final se ele já existir na lista (evita ler o próprio output)
    arquivos = [f for f in arquivos if "dataset_treino_final" not in f]
    
    print(f"📂 Encontrados {len(arquivos)} arquivos em {diretorio_raiz}")

    for arquivo in tqdm(arquivos, desc="Amostrando Dados"):
        try:
            # Lê o arquivo Parquet (muito rápido!)
            df_temp = pd.read_parquet(arquivo)
            
            # 1. Renomear colunas para padrão único
            df_temp.rename(columns=column_mapping, inplace=True)
            
            # 2. Remover espaços dos nomes das colunas (ex: " Label" -> "Label")
            df_temp.columns = df_temp.columns.str.strip()
            
            # 3. Tratamento básico de Label (padronizar maiúsculas/minúsculas)
            if 'Label' in df_temp.columns:
                df_temp['Label'] = df_temp['Label'].astype(str).str.strip()
            
            # 4. AMOSTRAGEM ALEATÓRIA
            # Pegamos apenas 10% deste arquivo de forma aleatória
            df_sample = df_temp.sample(frac=SAMPLE_RATE, random_state=42)
            
            # Adiciona colunas identificadoras (opcional, bom para rastreio)
            df_sample['origem_arquivo'] = os.path.basename(arquivo)
            
            dfs.append(df_sample)
            
        except Exception as e:
            print(f"❌ Erro ao ler {os.path.basename(arquivo)}: {e}")

    return dfs

# =============================================================================
# 3. EXECUÇÃO
# =============================================================================
print(f">>> INICIANDO CRIAÇÃO DO DATASET DE TREINO (Amostra: {SAMPLE_RATE*100}%) <<<")

# Carrega listas de DataFrames
dados_2017 = carregar_e_amostrar(os.path.join(BASE_DIR, "CICIDS2017"))
dados_2019 = carregar_e_amostrar(os.path.join(BASE_DIR, "CICDDoS2019")) # Pega Treino e Teste

# Junta tudo em uma lista única
todos_dfs = dados_2017 + dados_2019

if todos_dfs:
    print("\n🔄 Concatenando dados (Unindo 2017 + 2019)...")
    # Concatena ignorando índices antigos (reset_index)
    df_final = pd.concat(todos_dfs, axis=0, ignore_index=True)
    
    print(f"📊 Dimensões Finais: {df_final.shape[0]} linhas x {df_final.shape[1]} colunas")
    
    # Salva o arquivo final consolidado
    print(f"💾 Salvando em: {OUTPUT_FILE} ...")
    df_final.to_parquet(OUTPUT_FILE, compression='snappy')
    
    print("\n✅ SUCESSO! Dataset de treino criado.")
    print("Próximo passo: Análise Exploratória e Treinamento do Modelo.")

else:
    print("\n❌ ERRO: Nenhum dado foi carregado. Verifique os caminhos.")

>>> INICIANDO CRIAÇÃO DO DATASET DE TREINO (Amostra: 10.0%) <<<
📂 Encontrados 8 arquivos em E:\Estudos_Cybersecurity\Datasets_Cybersecurity\CICIDS2017


Amostrando Dados: 100%|██████████████████████████████████████████████████████████████████| 8/8 [00:11<00:00,  1.39s/it]


📂 Encontrados 18 arquivos em E:\Estudos_Cybersecurity\Datasets_Cybersecurity\CICDDoS2019


Amostrando Dados:  11%|███████▏                                                         | 2/18 [00:28<04:22, 16.41s/it]

❌ Erro ao ler TFTP.parquet: malloc of size 1048576 failed


...

❌ Erro ao ler DrDoS_UDP.parquet: 


Amostrando Dados:  33%|█████████████████████▋                                           | 6/18 [01:55<04:40, 23.39s/it]

❌ Erro ao ler DrDoS_SNMP.parquet: malloc of size 1073741824 failed


Amostrando Dados: 100%|████████████████████████████████████████████████████████████████| 18/18 [07:16<00:00, 24.27s/it]



🔄 Concatenando dados (Unindo 2017 + 2019)...
📊 Dimensões Finais: 4326968 linhas x 88 colunas
💾 Salvando em: E:\Estudos_Cybersecurity\Datasets_Cybersecurity\dataset_treino_final.parquet ...

✅ SUCESSO! Dataset de treino criado.
Próximo passo: Análise Exploratória e Treinamento do Modelo.
